# Region Generation

This notebok will separate the data into all the updates for a particular project.

Take the tile_placements data and separate it out into frames. A frame is a subset of the data.
The data can be split up into frames based on time (e.g. 1 frame is 30 minutes of data) or by number of updates (e.g. 1 frame is 1 million updates). Every update can only belong to one frame.
After creating the frames, create a graph where every pixel is a node. A single pixel will be a vector of all the different updates that happened within that one frame.

We want to do a min-cut on the graph so that every graph partition represents one image. To select the edge weights, we want edges between pixels within the same image to have a large weight and edges between pixels of different images should have small weights. 

After we do graph partitions within one frame, we want to connect the frames together. 


The ultimate goal is to connect the frames that hold all the updates for a single project and to train a CNN on this data

## Progress Updates:
##### May 15, 2019  

To start out, we will try to split based on updates because there is a large surge of updates near the end, so if we split by time, then the frames near the end will have significantly more updates than frames in the beginning.
There are 16 million datapoints, so we will split 100,000 updates per frame, which will result in 160 frames.
Frames will be stored as CSV files into the folder ../data/frames

##### May 17, 2019

Heirustics: Same color and same user should have a weight of 0. 

In [4]:
import csv
import numpy as np
import networkx as nx

In [22]:
'''
    If fixed_framesize is True, then the each frame will contain the same number of updates equal to the framesize parameter
    The frametime parameter will be ignored.
    Default framesize is 1 million updates
    
    If fixed_framesize is False, then each frame will specify a certain timespan. The duration of one frame is the frametime parameter.
    The framesize parameter will be ignored.
    Default frametime is 3 hours (10800 seconds)
'''
def create_frames( fixed_framesize = True, framesize = 1000000, frametime = 10800, filename = "../data/tile_placements.csv"):
    # All the frames will be stored in data/frames
    filecount = 0

    num_updates = 0
    oldest_time = None
    newest_time = None
    with open(filename) as f:
        # Skip the header row
        next(f, None)
        reader = csv.reader(f)
        for r in reader:
            num_updates += 1
            time = int(r[0])
            
            if oldest_time == None or time < oldest_time:
                oldest_time = time
            if newest_time == None or time > newest_time:
                newest_time = time
            


    frames = list()

    num_updates -= 1 # Subtract one to account for the header
    print("Num updates: ", num_updates)
    with open(filename,'r') as file_in:

        # Skip the header row
        next(file_in, None)
        reader = csv.reader(file_in)
        rows = list(reader)
        
        if (fixed_framesize):
            while (filecount < int(num_updates / framesize) + 1 ):

                frames.append([])

                for i in range(filecount * framesize, (filecount * framesize) + framesize):
                    if (i < num_updates):
                        frames[filecount].append(rows[i])

                filecount += 1
        else:
            timespan = newest_time - oldest_time
            num_frames = int(timespan / frametime) + 1
            for i in range(num_frames):
                frames.append([])
            
            for update in rows:
                frames_index = int((int(update[0]) - oldest_time)/timespan)
                frames[frames_index].append(update)
                
            
    frames = np.array(frames)
    print("DONE")
    return frames


In [17]:
# Create a graph where every pixel within a frame has an edge to its r nearest neighbors
def create_graph(frames, r = 1):
    
    num_lines = 0
    
    # Create a list of networkx Graphs. One for every frame. 
    # Each pixel will be a node and there will be an edge between that node and its r nearest neighbors
    graphs = list()
    
    for frame in frames:
        # First, parse through the frame and create a bunch of vectors, where each vector represents a pixel.
        # Each element of the vector represents one update to the pixel
        # Format of the pixels dictionary:
        '''
            {
                (x, y) : ("ts", "user", "color")
            }
        '''
        pixels = dict()
        
        for pixel in frame:
            ts = pixel[0]
            user = pixel[1]
            x = int(pixel[2])
            y = int(pixel[3])
            color = pixel[4]
        
            if (pixels.get((x, y)) == None):
                pixels[(x,y)] = list()
            
            pixels[(x,y)].append((ts, user, color))
   
    
        # Now, place all the pixels in a graph where there is an edge between every pixel and its neighbors that are r away
        G = nx.Graph()
        for coordinates in pixels:
            G.add_node(coordinates)

        for (x,y) in pixels:
            for distance in range(1, r + 1):
                if pixels.get((x+distance,y)) != None:
                    G.add_edge((x,y), ((x+1,y)))

                if pixels.get((x,y+distance)) != None:
                    G.add_edge((x,y), ((x,y+1)))

                if pixels.get((x-distance,y)) != None:
                    G.add_edge((x,y), ((x-1,y)))

                if pixels.get((x,y-distance)) != None:
                    G.add_edge((x,y), ((x,y-1)))    
    
        # Each networkx graph will be stored as a tuple
        # The first element is the graph. The second element is a dictionary containing data about each pixel
        graphs.append((G, pixels))
    
    return graphs

We will now add weights to all the edges.
Pixels that are in the same image should have larger weights.
Pixels that are in different images should have smaller weights.

Each x,y cooridnate has many updates placed there.
We first tried assigning weights based on most recent color.

In [ ]:

def add_edge_weights(graphs):
    for (graph, pixels_dict) in graphs:
        all_edges = list(graph.edges())
        for edge in all_edges:
            node1 = edge[0]
            node2 = edge[1]
            
            
            
            
        

In [19]:
frames = create_frames()
G = create_graph(frames)

Num updates:  16559896


In [23]:
frames2 = create_frames(fixed_framesize = False)
G2 = create_graph(frames2)

Num updates:  16559896
DONE
